In [ ]:
from matplotlib import pylab as plt
import matplotlib.dates as dates_2020
import pandas as pd
import seaborn as sns

In [ ]:
data = pd.read_csv("./bream_bay.csv")
data.head(2)

,DATE,TIME,Temperature,Day
0,1979-01-01,00:00:00,282.88965,0
1,1979-01-01,01:00:00,283.02603,0


In [ ]:
data["DATE_TIME"] = pd.to_datetime(data.DATE  + " " + data["TIME"])

In [ ]:
data.head(5)

,DATE,TIME,Temperature,Day,DATE_TIME
0,1979-01-01,00:00:00,282.88965,0,1979-01-01 00:00:00
1,1979-01-01,01:00:00,283.02603,0,1979-01-01 01:00:00
2,1979-01-01,02:00:00,283.13562,0,1979-01-01 02:00:00
3,1979-01-01,03:00:00,282.95294,0,1979-01-01 03:00:00
4,1979-01-01,04:00:00,282.92370,0,1979-01-01 04:00:00


In [ ]:
for i in range(1,6):
    data["T_" + str(i)] = data.Temperature.shift(i)

In [ ]:
data.fillna(0.00,inplace=True)
data.head(10)

,DATE,TIME,Temperature,Day,DATE_TIME,T_1,T_2,T_3,T_4,T_5
0,1979-01-01,00:00:00,282.88965,0,1979-01-01 00:00:00,0.00000,0.00000,0.00000,0.00000,0.00000
1,1979-01-01,01:00:00,283.02603,0,1979-01-01 01:00:00,282.88965,0.00000,0.00000,0.00000,0.00000
2,1979-01-01,02:00:00,283.13562,0,1979-01-01 02:00:00,283.02603,282.88965,0.00000,0.00000,0.00000
3,1979-01-01,03:00:00,282.95294,0,1979-01-01 03:00:00,283.13562,283.02603,282.88965,0.00000,0.00000
4,1979-01-01,04:00:00,282.92370,0,1979-01-01 04:00:00,282.95294,283.13562,283.02603,282.88965,0.00000
5,1979-01-01,05:00:00,282.90910,1,1979-01-01 05:00:00,282.92370,282.95294,283.13562,283.02603,282.88965
6,1979-01-01,06:00:00,282.45352,1,1979-01-01 06:00:00,282.90910,282.92370,282.95294,283.13562,283.02603
7,1979-01-01,07:00:00,282.10270,1,1979-01-01 07:00:00,282.45352,282.90910,282.92370,282.95294,283.13562
8,1979-01-01,08:00:00,282.19528,1,1979-01-01 08:00:00,282.10270,282.45352,282.90910,282.92370,282.95294
9,1979-01-01,09:00:00,282.22452,1,1979-01-01 09:00:00,282.19528,282.10270,282.45352,282.90910,282.92370


In [ ]:
clean_data = data[['Day', 'T_1','T_2', 'T_3', 'T_4', 'T_5','Temperature']]

In [ ]:
clean_data.head(2)

,Day,T_1,T_2,T_3,T_4,T_5,Temperature
0,0,0.00000,0.0,0.0,0.0,0.0,282.88965
1,0,282.88965,0.0,0.0,0.0,0.0,283.02603


In [ ]:
training_data = data[data.DATE_TIME < pd.to_datetime("01/01/2015")]

In [ ]:
training_data.tail(3)

,DATE,TIME,Temperature,Day,DATE_TIME,T_1,T_2,T_3,T_4,T_5
315573,2014-12-31,21:00:00,283.14084,0,2014-12-31 21:00:00,283.18637,283.37387,283.64978,283.70068,283.81320
315574,2014-12-31,22:00:00,282.21133,0,2014-12-31 22:00:00,283.14084,283.18637,283.37387,283.64978,283.70068
315575,2014-12-31,23:00:00,282.23007,0,2014-12-31 23:00:00,282.21133,283.14084,283.18637,283.37387,283.64978


In [ ]:
val_data = data[(data.DATE_TIME >= pd.to_datetime("01/01/2015")) & (data.DATE_TIME < pd.to_datetime("01/01/2017"))]

In [ ]:
test_data = data[data.DATE_TIME >= pd.to_datetime("01/01/2017")]

In [ ]:
test_data.head(3)

,DATE,TIME,Temperature,Day,DATE_TIME,T_1,T_2,T_3,T_4,T_5
333120,2017-01-01,00:00:00,281.61188,0,2017-01-01 00:00:00,281.47900,281.15317,280.29532,279.85464,279.65543
333121,2017-01-01,01:00:00,281.79382,0,2017-01-01 01:00:00,281.61188,281.47900,281.15317,280.29532,279.85464
333122,2017-01-01,02:00:00,281.85373,0,2017-01-01 02:00:00,281.79382,281.61188,281.47900,281.15317,280.29532


In [ ]:
val_data.tail(3)

,DATE,TIME,Temperature,Day,DATE_TIME,T_1,T_2,T_3,T_4,T_5
333117,2016-12-31,21:00:00,280.29532,0,2016-12-31 21:00:00,279.85464,279.65543,279.74683,279.86868,280.03275
333118,2016-12-31,22:00:00,281.15317,0,2016-12-31 22:00:00,280.29532,279.85464,279.65543,279.74683,279.86868
333119,2016-12-31,23:00:00,281.47900,0,2016-12-31 23:00:00,281.15317,280.29532,279.85464,279.65543,279.74683


In [ ]:
clean_train = training_data[['T_1','T_2', 'T_3', 'T_4', 'T_5','Temperature']]
clean_test = test_data[[ 'T_1','T_2', 'T_3', 'T_4', 'T_5','Temperature']]
clean_val = val_data[['T_1','T_2', 'T_3', 'T_4', 'T_5','Temperature']]

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
X_train,y_train = clean_train.drop(["Temperature"],axis=1),clean_train.Temperature
X_test,y_test = clean_test.drop(["Temperature"],axis=1),clean_test.Temperature
X_val,y_val = clean_val.drop(["Temperature"],axis=1),clean_val.Temperature

In [ ]:
scaler = StandardScaler()
#scaler = MinMaxScaler(feature_range=(-1, 1))
rfr  = RandomForestRegressor(random_state=None,verbose=2,n_jobs=20)

In [ ]:
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.fit_transform(X_test)
#X_valid_scaled = scaler.fit_transform(X_val)

In [ ]:
rfr.fit(X_train,y_train)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100building tree 10 of 100

building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100building tree 15 of 100

building tree 16 of 100
building tree 17 of 100
building tree 18 of 100building tree 19 of 100

building tree 20 of 100


[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:   38.7s


building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100building tree 33 of 100building tree 34 of 100


building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100building tree 54 of 100

building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62

[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:  3.3min finished


RandomForestRegressor(n_jobs=20, verbose=2)

In [ ]:
rfr.score(X_val,y_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    1.0s finished


0.9923107565861989

In [ ]:
rfr.score(X_test,y_test)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    1.0s finished


0.9921205469592764

In [ ]:
test_data["TEMP_PREDICTED"] = rfr.predict(X_test)
ypred = rfr.predict(X_test)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    1.0s finished
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    1.0s finished


In [ ]:
ypred

array([281.7395902, 281.4161246, 281.9410158, ..., 287.341715 ,
       287.268574 , 286.0302037])

In [ ]:
#test_data
mean_squared_error(y_test, ypred, squared=False)

0.3610268125436493

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral3
from bokeh.io import output_notebook
output_notebook()

In [ ]:
source = ColumnDataSource(test_data)

p = figure(x_axis_type='datetime')
p.line(x='DATE_TIME', y='Temperature', line_width=2, source=source, legend_label='Actual')
p.line(x='DATE_TIME', y='TEMP_PREDICTED', line_width=1, source=source, color=Spectral3[2], legend_label='Predicted')
p.yaxis.axis_label = 'Temperature'

show(p)

In [ ]:
clean_data2 = test_data[['DATE_TIME','Temperature','TEMP_PREDICTED']]
clean_data2

,DATE_TIME,Temperature,TEMP_PREDICTED
333120,2017-01-01 00:00:00,281.61188,281.765776
333121,2017-01-01 01:00:00,281.79382,281.361799
333122,2017-01-01 02:00:00,281.85373,281.856776
333123,2017-01-01 03:00:00,281.91360,281.863844
333124,2017-01-01 04:00:00,282.24295,281.973304
...,...,...,...
350635,2018-12-31 19:00:00,287.30786,287.231640
350636,2018-12-31 20:00:00,287.32630,287.287502
350637,2018-12-31 21:00:00,287.28946,287.301073
350638,2018-12-31 22:00:00,286.60776,287.238424


In [ ]:
clean_data2.to_csv("bb_sklearn.csv", index = False)